In [2]:
%%capture --no-stderr
%pip install -U langchain langchain_openai langsmith pandas langchain_experimental matplotlib langgraph langchain_core

### Define Agent

In [9]:
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import END, StateGraph, START


def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_tools(tools)

### Define Tools

In [10]:
from typing import Annotated
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL



In [19]:
from dotenv import load_dotenv
import requests
import time
import json
import os
import logging
import base64
import zlib

load_dotenv()

BASE_URL = "http://localhost:8100"
PDF_FILE_PATH = "../example_scripts/lacers_reduced.pdf"

def read_and_encode_pdf(file_path):
    with open(file_path, "rb") as file:
        pdf_content = base64.b64encode(zlib.compress(file.read())).decode('utf-8')
    logging.debug(f"{file_path} read and encoded")
    return pdf_content

def get_pipeline_results(task_id):
    logging.debug(f"Fetching results for task ID: {task_id}")
    response = requests.get(f"{BASE_URL}/pipelines/{task_id}")
    return response.json()

def process_pdf(pdf_file):
    pdf_content = read_and_encode_pdf(pdf_file)

    schema_1 = {
        "Firm": "The name of the firm",
        "Number of Funds": "The number of funds managed by the firm",
        "Commitment": "The commitment amount in millions of dollars",
        "Percent of Total Comm": "The percentage of total commitment",
        "Exposure (FMV + Unfunded)": "The exposure including fair market value and unfunded commitments in millions of dollars",
        "Percent of Total Exposure": "The percentage of total exposure",
        "TVPI": "Total Value to Paid-In multiple",
        "Net IRR": "Net Internal Rate of Return as a percentage"
    }

    pipeline_request = {
        "workloads": [
            {
                "pdf_stream": pdf_content,
                "schemas": [json.dumps(schema_1)]
            }
        ],
        "provider_type": "azure",
        "provider_model_name": os.getenv("AZURE_MODEL_NAME"),
        "api_key": os.getenv("AZURE_OPENAI_API_KEY"),
        "additional_params": {
            "azure_endpoint": os.getenv("AZURE_ENDPOINT"),
            "azure_deployment": os.getenv("AZURE_DEPLOYMENT_ID"),
            "api_version": os.getenv("AZURE_API_VERSION")
        }
    }

    logging.debug("Sending POST request to pipeline endpoint")
    try:
        response = requests.post(f"{BASE_URL}/pipelines", json=pipeline_request)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        logging.error(f"Error sending request: {e}")
        return

    logging.debug(f"Response status code: {response.status_code}")
    logging.debug(f"Response headers: {response.headers}")
    logging.debug(f"Response content: {response.text}")

    try:
        result = response.json()
    except json.JSONDecodeError:
        logging.error("Failed to decode JSON response")
        return

    task_id = result.get("task_id")
    if not task_id:
        logging.error("Invalid task_id: task_id is None or empty")
        return
    logging.debug(f"Task ID: {task_id}")

    max_attempts = 5
    attempt = 0
    while attempt < max_attempts:
        time.sleep(30)

        results = get_pipeline_results(task_id)
        logging.debug(f"Poll attempt {attempt + 1}: Status - {results['status']}")

        if results['status'] == 'COMPLETED':
            logging.debug(f"Pipeline completed with results: {results['results']}")
            return results['results']
        elif results['status'] == 'FAILED':
            logging.error(f"Error: {results.get('error_message', 'Unknown error')}")
            return None

        attempt += 1

    logging.warning("Timeout: Pipeline execution took too long.")
    return None

@tool("process_pdf")
def process_pdf(pdf_file_path: str) -> str:
    """Process a PDF file and return a string."""
    return process_pdf(pdf_file_path)


repl = PythonREPL()

@tool("python_repl")
def python_repl(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. """
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with FINAL ANSWER."
    )

### Define State

In [ ]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain_openai import ChatOpenAI


# This defines the object that is passed between each node
# in the graph. We will create different nodes for each agent and tool
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

### Define Agent Nodes